## This code is to save the pure documents from a subset of NQ, which is the CLAPQN dataset. In the end we can save 1790 preprocessed passages.

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

def explore_jsonl(file_path):
    # Load the JSONL file
    data = load_jsonl(file_path)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    print("Data Overview:")
    print(f"Number of records: {len(df)}")
    print(f"\nColumns: {', '.join(df.columns)}")

    print("\nData Types:")
    print(df.dtypes)

    print("\nSample Data:")
    print(df.head())

    print("\nSummary Statistics:")
    print(df.describe())

    return df

file_path = 'path_to_your_folder/clapnq/original_documents/train/clapnq_train_answerable_orig.jsonl'  # Replace with your JSONL file path
df = explore_jsonl(file_path)

Data Overview:
Number of records: 1791

Columns: document_plaintext, document_title, document_url, example_id, language, question_text, passage_answer_candidates, annotations

Data Types:
document_plaintext           object
document_title               object
document_url                 object
example_id                   object
language                     object
question_text                object
passage_answer_candidates    object
annotations                  object
dtype: object

Sample Data:
                                  document_plaintext  \
0  Cuisine of Equatorial Guinea - wikipedia Cuisi...   
1  List of Awards and nominations received by Mic...   
2  Monkey 's uncle - Wikipedia Monkey 's uncle Th...   
3  UNICEF - Wikipedia UNICEF United Nations Child...   
4  Year and a day rule - wikipedia Year and a day...   

                                      document_title  \
0                       Cuisine of Equatorial Guinea   
1  List of awards and nominations received by M

In [ ]:
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        return [json.loads(line) for line in file]

def explore_jsonl(file_path):
    # Load the JSONL file
    data = load_jsonl(file_path)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    print("Data Overview:")
    print(f"Number of records: {len(df)}")
    print(f"\nColumns: {', '.join(df.columns)}")

    print("\nData Types:")
    print(df.dtypes)

    print("\nSample Data:")
    print(df.head())

    print("\nSummary Statistics:")
    print(df.describe())

    return df

In [ ]:
file_path = 'path_to_your_folder/clapnq/original_documents/dev/clapnq_dev_answerable_orig.jsonl'  # Replace with your JSONL file path
df = explore_jsonl(file_path)

Data Overview:
Number of records: 300

Columns: document_plaintext, document_title, document_url, example_id, language, question_text, passage_answer_candidates, annotations

Data Types:
document_plaintext           object
document_title               object
document_url                 object
example_id                   object
language                     object
question_text                object
passage_answer_candidates    object
annotations                  object
dtype: object

Sample Data:
                                  document_plaintext  \
0  Daniel Johnston - wikipedia Daniel Johnston Fo...   
1  University of Alabama traditions - wikipedia U...   
2  Mr. Lawrence - wikipedia Mr. Lawrence This is ...   
3  Making out - wikipedia Making out Jump to : na...   
4  Lake Kariba - wikipedia Lake Kariba Jump to : ...   

                     document_title  \
0                   Daniel Johnston   
1  University of Alabama traditions   
2                      Mr. Lawrence   
3   

In [ ]:
file_path = 'path_to_your_folder/clapnq_dev_answerable.jsonl'  # Replace with your JSONL file path
df = explore_jsonl(file_path)

Data Overview:
Number of records: 300

Columns: id, input, passages, output

Data Types:
id          object
input       object
passages    object
output      object
dtype: object

Sample Data:
                     id                                              input  \
0   6401197308716204890  which method of forecasting uses averages to p...   
1  -1218875241352839456  difference between russian blue and british bl...   
2   7812627963583184440   what is the role of the prime minister in canada   
3   1249598914156603825  the latin phrase parens patriae was used to re...   
4   7663406429430503589         who were the dadaists and what did they do   

                                            passages  \
0  [{'title': 'Forecasting', 'text': 'Seasonality...   
1  [{'title': 'Russian Blue', 'text': 'The coat i...   
2  [{'title': 'Prime Minister of Canada', 'text':...   
3  [{'title': 'Parens patriae', 'text': 'Parens p...   
4  [{'title': 'Dada', 'text': 'Dada ( / ˈdɑːdɑː /...   

 

In [ ]:
import json
import csv
from tqdm import tqdm
import logging
import re
import pandas as pd

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Paths to your JSONL files
file1_path = 'Path to your folder/clapnq/annotated_data/dev/clapnq_dev_answerable.jsonl'
file2_path = 'Path to your folder/clapnq/original_documents/dev/clapnq_dev_answerable_orig.jsonl'

output_csv_path = 'Path to your folder/clapnq/subset200.csv'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def clean_wikipedia_text(text):
    """
    Clean Wikipedia text by removing unnecessary elements and formatting.

    Args:
        text: Input text to clean

    Returns:
        str: Cleaned text
    """
    # Remove everything after "See also", "References", or "External links"
    end_markers = ["See also ( edit )", "References ( edit )", "External links ( edit )", "Jump up ^"]
    for marker in end_markers:
        if marker in text:
            text = text.split(marker)[0]
            break

    # Remove editorial notes
    text = re.sub(r'\[edit\]', '', text)

    # Remove content within parentheses
    text = re.sub(r'\([^()]*\)', '', text)

    # Remove citations
    text = re.sub(r'\[\d+\]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

def process_jsonl_files(file1_path: str, file2_path: str, output_path: str, num_rows: int = 250):
    """
    Process two JSONL files and create a combined CSV with exactly num_rows of non-empty plaintext data.
    """
    # Read the first JSONL file
    logger.info("Reading first JSONL file...")
    file1_data = []
    with open(file1_path, 'r', encoding='utf-8') as f:
        for line in f:
            file1_data.append(json.loads(line.strip()))

    # Create a set of titles and get their plaintext from file2
    logger.info("Reading second JSONL file...")
    title_to_plaintext = {}
    with open(file2_path, 'r', encoding='utf-8') as f:
        for line in f:
            data = json.loads(line.strip())
            title_to_plaintext[data['document_title']] = clean_wikipedia_text(data['document_plaintext'])

    # Create the combined dataset
    logger.info("Creating combined dataset...")
    combined_data = []

    for item in file1_data:
        passage = item['passages']
        output = item['output']

        for doc in passage:
            plaintext = title_to_plaintext.get(doc['title'], '')
            combined_data.append({
                'id': item['id'],
                'input': item['input'],
                'title': doc['title'],
                'text': doc['text'],
                'answer': output[0]['answer'] if output else '',
                'plaintext': plaintext
            })

    # Convert to DataFrame
    df = pd.DataFrame(combined_data)

    # Filter out rows with empty plaintext and take first num_rows
    df_filtered = df[df['plaintext'].str.strip() != ''].head(num_rows)

    # Save to CSV
    df_filtered.to_csv(output_path, index=False)
    logger.info(f"Successfully saved {len(df_filtered)} complete rows to {output_path}")

    return df_filtered

In [ ]:
# Process the files
df = process_jsonl_files(file1_path, file2_path, output_csv_path)

# Optional: Display the first few rows of the result
display(df.head())

,id,input,title,text,answer,plaintext
0,6401197308716204890,which method of forecasting uses averages to p...,Forecasting,Seasonality is a characteristic of a time seri...,"Seasonality, in which data experiences regular...",Forecasting - wikipedia Forecasting Jump to : ...
1,-1218875241352839456,difference between russian blue and british bl...,Russian Blue,"The coat is known as a `` double coat '' , wit...","Russian Blue is with the undercoat being soft,...",Russian Blue - wikipedia Russian Blue For the ...
2,7812627963583184440,what is the role of the prime minister in canada,Prime Minister of Canada,"Because the prime minister is , in practice , ...",Prime Minister of Canada is responsible for ad...,Prime minister of Canada - Wikipedia Prime min...
3,1249598914156603825,the latin phrase parens patriae was used to re...,Parens patriae,Parens patriae is Latin for `` parent of the n...,"In law, Latin Parens patriae refers to the pub...",Parens patriae - wikipedia Parens patriae Jump...
4,7663406429430503589,who were the dadaists and what did they do,Dada,Dada ( / ˈdɑːdɑː / ) or Dadaism was an art mov...,Dadaist includes artists who rejected the logi...,"Dada - wikipedia Dada Jump to : navigation , s..."


In [ ]:
len(filtered_entries_second_file)

205

In [ ]:
filtered_entries_second_file[0].keys()

dict_keys(['document_plaintext', 'document_title', 'document_url', 'example_id', 'language', 'question_text', 'passage_answer_candidates', 'annotations'])

In [ ]:
filtered_entries_second_file[0]['annotations']

[{'annotation_id': '10556050442205767452',
  'minimal_answer': {'plaintext_start_byte': -1, 'plaintext_end_byte': -1},
  'passage_answer': {'candidate_index': 309},
  'yes_no_answer': 'NONE'}]

In [ ]:
filtered_entries_second_file[0]['document_plaintext']

"First battle of Bull Run - wikipedia First battle of Bull Run This is the latest accepted revision , reviewed on 31 July 2018 . First Battle of Bull Run First Battle of Manassas Part of the American Civil War First Battle of Bull Run . Chromolithograph by Kurz & Allison , 1889 Date July 21 , 1861 ( 1861 - 07 - 21 ) Location Fairfax County and Prince William County , Virginia 38 ° 48 ′ 53 '' N 77 ° 31 ′ 22 '' W \ufeff / \ufeff 38.8147 ° N 77.5227 ° W \ufeff / 38.8147 ; - 77.5227 Coordinates : 38 ° 48 ′ 53 '' N 77 ° 31 ′ 22 '' W \ufeff / \ufeff 38.8147 ° N 77.5227 ° W \ufeff / 38.8147 ; - 77.5227 Result Decisive Confederate victory Belligerents United States ( Union ) Confederate States ( Confederacy ) Commanders and leaders Political leaders : Abraham Lincoln Military commanders : Winfield Scott Irvin McDowell Robert Patterson Political leaders : Jefferson Davis Military commanders : P.G.T. Beauregard Thomas `` Stonewall '' Jackson ( WIA ) Joseph E. Johnston Units involved Department o

In [ ]:
import re

def clean_wikipedia_text(text):
    # Remove everything after "See also", "References", or "External links"
    end_markers = ["See also ( edit )", "References ( edit )", "External links ( edit )", "Jump up ^"]
    for marker in end_markers:
        if marker in text:
            text = text.split(marker)[0]
            break

    # Remove editorial notes
    text = re.sub(r'\[edit\]', '', text)

    # Remove content within parentheses
    text = re.sub(r'\([^()]*\)', '', text)

    # Remove citations
    text = re.sub(r'\[\d+\]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    return text.strip()

In [ ]:
raw_text = df.iloc[1000]["document_plaintext"]
cleaned_text = clean_wikipedia_text(raw_text)

# Now you can pass cleaned_text to your semantic splitter for chunking

In [ ]:
len(df)

1791

In [ ]:
cleaned_text

"The Turning Point - wikipedia The Turning Point Jump to : navigation , search This article is about an episode of TV series The Vampire Diaries . For other uses , see Turning Point . `` The Turning Point '' The Vampire Diaries episode Episode no . Season 1 Episode 10 Directed by J. Miller Tobin Story by Barbie Kligman Teleplay by Kevin Williamson Julie Plec Production code 2J5009 Original air date November 19 , 2009 Guest appearance Matt Davis Chris Johnson Marguerite MacIntyre Episode chronology ← Previous `` History Repeating '' Next → `` Bloodlines '' The Vampire Diaries List of The Vampire Diaries episodes `` The Turning Point '' is the tenth episode of the first season of The CW television series , The Vampire Diaries and the tenth episode of the series overall . It originally aired on November 19 , 2009 . The episode 's story was written by Barbie Kligman and the teleplay by Kevin Williamson and Julie Plec . It was directed by J. Miller Tobin . Contents 1 Plot 2 Feature music 3 

In [ ]:
raw_text = df.iloc[1000]["document_plaintext"]
raw_text

"The Turning Point ( the Vampire Diaries ) - wikipedia The Turning Point ( the Vampire Diaries ) Jump to : navigation , search This article is about an episode of TV series The Vampire Diaries . For other uses , see Turning Point . `` The Turning Point '' The Vampire Diaries episode Episode no . Season 1 Episode 10 Directed by J. Miller Tobin Story by Barbie Kligman Teleplay by Kevin Williamson Julie Plec Production code 2J5009 Original air date November 19 , 2009 Guest appearance ( s ) Matt Davis ( Alaric Saltzman ) Chris Johnson ( Logan Fell ) Marguerite MacIntyre ( Sheriff Forbes ) Episode chronology ← Previous `` History Repeating '' Next → `` Bloodlines '' The Vampire Diaries ( season 1 ) List of The Vampire Diaries episodes `` The Turning Point '' is the tenth episode of the first season of The CW television series , The Vampire Diaries and the tenth episode of the series overall . It originally aired on November 19 , 2009 . The episode 's story was written by Barbie Kligman and 

In [ ]:
import os

def process_and_save_texts(df, output_folder, text_column="document_plaintext"):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process only the first num_rows
    for index, row in df.iterrows():
        # Get the raw text
        raw_text = row[text_column]

        # Clean the text
        cleaned_text = clean_wikipedia_text(raw_text)

        # Create a filename
        filename = f"cleaned_text_{index}.txt"

        # Full path for the file
        file_path = os.path.join(output_folder, filename)

        # Write the cleaned text to a file
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(cleaned_text)

        print(f"Processed and saved: {filename}")

# Usage
output_folder = "path_to_your_folder/clapnq/cleandoc"
process_and_save_texts(df, output_folder)

Processed and saved: cleaned_text_0.txt
Processed and saved: cleaned_text_1.txt
Processed and saved: cleaned_text_2.txt
Processed and saved: cleaned_text_3.txt
Processed and saved: cleaned_text_4.txt
Processed and saved: cleaned_text_5.txt
Processed and saved: cleaned_text_6.txt
Processed and saved: cleaned_text_7.txt
Processed and saved: cleaned_text_8.txt
Processed and saved: cleaned_text_9.txt
Processed and saved: cleaned_text_10.txt
Processed and saved: cleaned_text_11.txt
Processed and saved: cleaned_text_12.txt
Processed and saved: cleaned_text_13.txt
Processed and saved: cleaned_text_14.txt
Processed and saved: cleaned_text_15.txt
Processed and saved: cleaned_text_16.txt
Processed and saved: cleaned_text_17.txt
Processed and saved: cleaned_text_18.txt
Processed and saved: cleaned_text_19.txt
Processed and saved: cleaned_text_20.txt
Processed and saved: cleaned_text_21.txt
Processed and saved: cleaned_text_22.txt
Processed and saved: cleaned_text_23.txt
Processed and saved: clean

## Chunk estimation

In [ ]:
import os
import random
from tqdm import tqdm

def calculate_total_length(directory, sample_size=200):
    all_files = [f for f in os.listdir(directory) if f.endswith('.txt')]
    sampled_files = random.sample(all_files, min(sample_size, len(all_files)))

    total_length = 0
    for filename in tqdm(sampled_files, desc="Processing documents"):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            total_length += len(content)

    avg_length = total_length / len(sampled_files)
    estimated_total = avg_length * len(all_files)

    return estimated_total, len(all_files)

# Usage
directory_path = "/content/drive/MyDrive/Thesis_LCT/Datasets/clapnq/cleandoc"
estimated_total, doc_count = calculate_total_length(directory_path)

print(f"Estimated total length of all documents: {estimated_total:.0f} characters")
print(f"Number of documents: {doc_count}")
print(f"Estimated average document length: {estimated_total / doc_count:.2f} characters")
print(f"Estimated average chunks per document: {(estimated_total / doc_count) / 512:.2f}")

Processing documents: 100%|██████████| 200/200 [00:49<00:00,  4.01it/s]

Estimated total length of all documents: 53764459 characters
Number of documents: 1791
Estimated average document length: 30019.24 characters
Estimated average chunks per document: 58.63


In [ ]:
import os
from math import ceil
from tqdm import tqdm

def estimate_chunks(directory, chunk_size=512, sample_size=200):
    all_files = [f for f in os.listdir(directory) if f.endswith('.txt')]  # Adjust if you have different file types
    sampled_files = all_files if len(all_files) <= sample_size else random.sample(all_files, sample_size)

    total_chunks = 0
    total_chars = 0

    for filename in tqdm(sampled_files, desc="Processing documents"):
        file_path = os.path.join(directory, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        chars = len(text)
        chunks = ceil(chars / chunk_size)

        total_chunks += chunks
        total_chars += chars

    avg_chunks = total_chunks / len(sampled_files)
    avg_chars = total_chars / len(sampled_files)

    estimated_total_chunks = avg_chunks * len(all_files)
    estimated_total_chars = avg_chars * len(all_files)

    return estimated_total_chunks, estimated_total_chars, len(all_files)

# Usage
directory_path = "/content/drive/MyDrive/Thesis_LCT/Datasets/clapnq/cleandoc"
chunk_size = 512  # Make sure this matches your actual chunk_size

est_chunks, est_chars, doc_count = estimate_chunks(directory_path, chunk_size)

print(f"Estimated total number of chunks: {est_chunks:.0f}")
print(f"Estimated total characters: {est_chars:.0f}")
print(f"Number of documents: {doc_count}")
print(f"Estimated average chunks per document: {est_chunks / doc_count:.2f}")
print(f"Estimated average characters per document: {est_chars / doc_count:.2f}")

Processing documents: 100%|██████████| 200/200 [00:27<00:00,  7.15it/s]

Estimated total number of chunks: 110218
Estimated total characters: 55964156
Number of documents: 1791
Estimated average chunks per document: 61.54
Estimated average characters per document: 31247.44


In [ ]:
text = "McMillen quietly released the game to Steam for personal computers , where it proved to be very popular . Looking to expand the game , McMillen and Himsl discovered limitations on Flash that made expansion difficult . Though they were able to incorporate more content through the Wrath of the Lamb expansion , McMillen had to forego a second planned expansion due to the Flash limitations . After the release of the original game , McMillen was approached by Tyrone Rodriguez of Nicalis , a development and publishing studio that had helped to bring personal computer games Cave Story and VVVVVV to consoles . Rodriguez offered Nicalis ' services to help port The Binding of Isaac to consoles . McMillen was interested but required that they needed to recreate the game outside of Flash to incorporate the additional material he had to forego previously , and to fix some additional bugs found since release ."

In [ ]:
len(text)

909

In [ ]:
text = """<CoT>:
1. Identify key information: The context discusses the application of Newton's laws of motion in a rotating frame and mentions fictitious forces like centrifugal force.
2. Locate relevant quote: ##begin_quote##The centrifugal force and other fictitious forces must be included along with the real forces in order to apply Newton 's laws of motion in the rotating frame##end_quote##.
3. Analyze the quote: This indicates that in a rotating frame, fictitious forces are necessary to accurately apply Newton's laws, as they account for the discrepancies observed when only real forces are considered.
4. Summary: The context clearly states that fictitious forces, such as centrifugal force, are essential for the correct application of Newton's laws in a rotating frame, as they help explain the behavior of objects that would otherwise seem to contradict these laws.

<ANSWER>: Fictitious forces, like centrifugal force, are essential for applying Newton's laws of motion in a rotating frame, as they account for the observed behavior of objects that do not accelerate as expected."""

In [ ]:
len(text)

1086